In [5]:
import pandas as pd
import numpy as np
import PyPDF2
import textract
import re

In [6]:
filename ='Syllabus_CSC462_AI.pdf' 

pdfFileObj = open(filename,'rb')               #open allows you to read the file
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   #The pdfReader variable is a readable object that will be parsed
num_pages = pdfReader.numPages                 #discerning the number of pages will allow us to parse through all the pages


count = 0
text = ""
                                                            
while count < num_pages:                       #The while loop will read each page
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
    
#Below if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.

if text != "":
    text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text

else:
    text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

    # Now we have a text variable which contains all the text derived from our PDF file.

In [12]:
text = text.encode().decode("ascii")

In [13]:
text = text.encode().decode('ascii','ignore').lower() #Lowercasing each word
type(text)

str

In [14]:
keywords = re.findall(r'[a-zA-Z]\w+',text)
len(keywords)                               #Total keywords in document

943

In [15]:
df = pd.DataFrame(list(set(keywords)),columns=['keywords'])  #Dataframe with unique keywords to avoid repetition in rows

In [16]:
def weightage(word,text,number_of_documents=1):
    word_list = re.findall(word,text)
    number_of_times_word_appeared =len(word_list)
    tf = number_of_times_word_appeared/float(len(text))
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return number_of_times_word_appeared,tf,idf ,tf_idf

In [17]:
df['number_of_times_word_appeared'] = df['keywords'].apply(lambda x: weightage(x,text)[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,text)[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,text)[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,text)[3])

In [18]:
df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(25)

,keywords,number_of_times_word_appeared,tf,idf,tf_idf
260,in,113,0.012800,-4.727388,-0.060511
409,al,77,0.008722,-4.343805,-0.037888
58,nt,76,0.008609,-4.330733,-0.037283
368,on,74,0.008382,-4.304065,-0.036078
13,or,71,0.008043,-4.262680,-0.034283
199,ch,66,0.007476,-4.189655,-0.031323
108,ion,52,0.005890,-3.951244,-0.023274
424,an,43,0.004871,-3.761200,-0.018320
211,at,42,0.004758,-3.737670,-0.017782
413,he,41,0.004644,-3.713572,-0.017247


In [19]:
from gensim.summarization import keywords
import warnings
warnings.filterwarnings("ignore")

values = keywords(text=text,split='\n',scores=True)

data = pd.DataFrame(values,columns=['keyword','score'])
data = data.sort_values('score',ascending=False)
data.head(10)



,keyword,score
0,sec,0.285705
2,courses,0.265973
1,course,0.265973
3,agent,0.247123
4,agents,0.247123
5,search,0.246018
6,learning,0.187905
7,problem,0.136409
9,lecture,0.135459
8,lectures,0.135459


In [20]:
from rake_nltk import Rake

r = Rake()
r.extract_keywords_from_text(text)

phrases = r.get_ranked_phrases_with_scores()

table = pd.DataFrame(phrases,columns=['score','Phrase'])
table = table.sort_values('score',ascending=False)
table.head(10)

,score,Phrase
0,74.966667,course prerequisites course code title csc 102...
1,64.464713,5 2 9 5 multilayer feed forward neural net wor...
2,49.763293,2 5 sessional 2 week 22 6 assignment 4
3,45.141026,1 2 sessional 1 week 11 3 assignment 2
4,41.500000,use c urrent tools n ece ssary
5,39.960394,2 2 6 5 learning decision trees ch
6,39.833333,single layer feed forward neural net
7,38.000000,artifact due date remarks 1 assignment 1
8,37.345010,3 27 5 learning decision trees ch
9,36.163768,4 22 4 backward chaining algorithm ch
